<img src="https://upload.wikimedia.org/wikipedia/commons/4/47/Logo_UTFSM.png" width="200" alt="utfsm-logo" align="left"/>

# MAT281
### Aplicaciones de la Matemática en la Ingeniería

## Módulo 02
## Laboratorio Clase 04: Agrupando datos

### Instrucciones


* Completa tus datos personales (nombre y rol USM) en siguiente celda.
* La escala es de 0 a 4 considerando solo valores enteros.
* Debes _pushear_ tus cambios a tu repositorio personal del curso.
* Como respaldo, debes enviar un archivo .zip con el siguiente formato `mXX_cYY_lab_apellido_nombre.zip` a alonso.ogueda@gmail.com. 
* Se evaluará:
    - Soluciones
    - Código
    - Que Binder esté bien configurado.
    - Al presionar  `Kernel -> Restart Kernel and Run All Cells` deben ejecutarse todas las celdas sin error.
* __La entrega es al final de esta clase.__

__Nombre__: Bernardo Recabarren Beltrán

__Rol__: 201610004-0

Se utilizará el mismo dataset de pokemon

In [1]:
import os
import pandas as pd

In [2]:
pkm = (
    pd.read_csv(os.path.join("data", "pokemon.csv"), index_col="#")
    .rename(columns=lambda x: x.replace(" ", "").replace(".", "_").lower())
)
pkm.head()

,name,type1,type2,hp,attack,defense,sp_atk,sp_def,speed,generation,legendary
#,,,,,,,,,,,
1,Bulbasaur,Grass,Poison,45,49,49,65,65,45,1,False
2,Ivysaur,Grass,Poison,60,62,63,80,80,60,1,False
3,Venusaur,Grass,Poison,80,82,83,100,100,80,1,False
4,Mega Venusaur,Grass,Poison,80,100,123,122,120,80,1,False
5,Charmander,Fire,NaN,39,52,43,60,50,65,1,False


## Ejercicio #1 (1 pto)

Agrupar por `generation` y `legendary` y obtener por grupo:

* Promedio de `hp`
* Mínimo y máximo de `sp_atk` y `sp_def`

In [3]:
(
    pkm.groupby(["generation","legendary"])
    .agg(
        sp_atk_min=("sp_atk","min"),
        sp_atk_max=("sp_atk","max"),
        sp_def_min=("sp_def","min"),
        sp_def_max=("sp_def","max"),
    )
)

sp_atk_min  sp_atk_max  sp_def_min  sp_def_max
generation legendary                                                
1          False              15         175          20         130
           True               95         194          85         125
2          False              10         165          20         230
           True               90         115          75         154
3          False              10         165          20         135
           True               50         180          20         200
4          False              10         140          30         150
           True               75         150          70         130
5          False              15         145          25         135
           True               72         170          72         129
6          False              27         150          25         154
           True               81         170          90         150

## Ejercicio #2 (1 pto)

El profesor Oakgueda determinó que una buen indicador de pokemones es: 

$$ 0.2 \, \textrm{hp} + 0.4 \,(\textrm{attack} + \textrm{sp_atk})^2 + 0.3 \,( \textrm{defense} + \textrm{sp_deff})^{1.5} + 0.1 \, \textrm{speed}$$

Según este indicador, ¿Qué grupo de pokemones (`type1`, `type2`) es en promedio mejor que el resto? 

In [4]:
def oakgueda_indicator(df):
    oakgueda_ind = 0.2*df["hp"] + 0.4*(df["attack"] + df["sp_atk"])**2 + 0.3*(df["defense"] + df["sp_def"])**1.5 + 0.1*df["speed"]
    return oakgueda_ind.mean()

In [5]:
pkm.groupby(["type1","type2"]).apply(oakgueda_indicator).idxmax() #Googlie la función argmax para panda

('Ground', 'Fire')

__Respuesta__: En promedio, la mejor combinación de tipos es Ground-Fire.

## Ejercicio #3 (1 pto)

Define una función que escale los datos tal que, si $s$ es una columna: 

$$s\_scaled = \frac{s - \min(s)}{\max(s) - \min(s)}$$

Y luego transforma cada columna agrupando por si el pokemon es legendario o no.

In [6]:
def minmax_scale(s):
    return (s -s.min())/float((s.max()- s.min()))

pkm.groupby(["legendary"]).transform(minmax_scale)

,hp,attack,defense,sp_atk,sp_def,speed,generation
#,,,,,,,
1,0.173228,0.244444,0.195556,0.333333,0.214286,0.258065,0.0
2,0.232283,0.316667,0.257778,0.424242,0.285714,0.354839,0.0
3,0.311024,0.427778,0.346667,0.545455,0.380952,0.483871,0.0
4,0.311024,0.527778,0.524444,0.678788,0.476190,0.483871,0.0
5,0.149606,0.261111,0.168889,0.303030,0.142857,0.387097,0.0
...,...,...,...,...,...,...,...
796,0.000000,0.357143,0.722222,0.347222,0.722222,0.000000,1.0
797,0.000000,0.785714,0.500000,0.763889,0.500000,0.461538,1.0
798,0.300000,0.428571,0.222222,0.694444,0.611111,0.153846,1.0


## Ejercicio #4 (1 pto)

El profesor Oakgueda necesita saber cuántos pokemones hay luego de filtrar el dataset tal que el grupo de (`type1`, `type2`) tenga en promedio un indicador (el del ejercicio #2) mayor a 50000.

In [7]:
pkm.groupby(["type1","type2"]).filter(lambda df: oakgueda_indicator(df) > 40000) 
#supuse que era 40k, ya que el máximo es de 44k

,name,type1,type2,hp,attack,defense,sp_atk,sp_def,speed,generation,legendary
#,,,,,,,,,,,
425,Primal Groudon,Ground,Fire,100,180,160,150,90,90,3,True
799,Hoopa Unbound,Psychic,Dark,80,160,60,170,130,80,6,True


__Respuesta:__ Hay solo dos pokemones que cumplen esta condición.